[twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


In [1]:
import requests
import base64
from copy import deepcopy
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from textblob import TextBlob
from bs4 import BeautifulSoup as bsoup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nltk
import re
import ast
import emoji

In [2]:
#Makarands Keys (remember to remove)

client_key = "uDxiPfIPIcFjGD0fmxj7c2XAk"
client_secret = "Bcr55JD2dNhGFftvtWwn19x78pRPLNP5rCBmGw0yVeW5Uy3hpc" 

In [3]:
key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [4]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [5]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [6]:
json_data =  response.json()
json_data

{'token_type': 'bearer',
 'access_token': 'AAAAAAAAAAAAAAAAAAAAAEskIQEAAAAAqkugIbkkPuB3XUUBhHAYkbYX4ik%3DjxVjLVRb5O9OUZ95hy8Z3cDHaTkA4j7VJM0Shx3GNLq2W8SdOA'}

In [7]:
access_token = json_data['access_token']

# Get Tweets
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [8]:
max_tweets=200 #CHANGED TO 200 FOR TESTING - MUST BE at least 1000 for analysis
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []

In [9]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

keywords = ["Packers","Bears","Eagles", "Cowboys","Raiders", "Chiefs"]

for keyword in keywords:
    #parameters popular and extended tweet mode
    parameters = { 'q': keyword,
                        'result_type': 'mixed',
                    'count': tw_limit, 
                     'tweet_mode': 'extended'}

    search_url = base_url+'1.1/search/tweets.json'

    response = requests.get(search_url, headers=search_headers, params=parameters)
    football_tweets = response.json()

    for status in football_tweets['statuses']:
        status["team"] = keyword
    tweets += football_tweets['statuses']
    ids = [tw['id'] for tw in football_tweets['statuses']]
    min_id = min(ids)
    tw_ids = []
    tw_ids+=ids

    for  i in range(max_tweets//tw_limit -1):
        parameters = { 'q': keyword,
                        'result_type': 'mixed',
                    'count': tw_limit,
                    'max_id': min(tw_ids)-1
                     }
        print("searching tweets with id  < {}".format(min_id))
        search_url = base_url+'1.1/search/tweets.json'
        response = requests.get(search_url, headers=search_headers, params=parameters)
        football_tweets = response.json()
        ids = [tw['id'] for tw in football_tweets['statuses']]
        tw_ids+=ids
        print(i, len(football_tweets['statuses']))
        min_id = min(ids)
        for status in football_tweets['statuses']:
            status["team"] = keyword
        tweets += football_tweets['statuses']    

searching tweets with id  < 1327317809945325569
0 100
searching tweets with id  < 1327365747203584002
0 100
searching tweets with id  < 1327149850664853504
0 100
searching tweets with id  < 1327235132001017858
0 100
searching tweets with id  < 1327151111883591680
0 100
searching tweets with id  < 1327041422810296321
0 100


As per twitter api documentation we can use

max_id:Returns results with an ID less than (that is, older than) or equal to the specified ID.

to get older tweets

# Q1 (.5 points) Modify only *parameters* dictionary in the following code so that subsequent call to climate change search returns older tweets with id less than minimum of previous tweets ids.

In [10]:
print('Total tweets retrieved= {}'.format(len(tweets)))

Total tweets retrieved= 1200


# Let's print some tweets

In [11]:
import pprint #pretty-print” arbitrary Python data structures. Tweets object has hierarchical struture
max_tweet_to_print=4

In [12]:
pprint.pprint(tweets[0:max_tweet_to_print])

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Fri Nov 13 21:27:20 +0000 2020',
  'display_text_range': [0, 212],
  'entities': {'hashtags': [{'indices': [182, 190], 'text': 'Packers'}],
               'media': [{'display_url': 'pic.twitter.com/gooxAF2Pkc',
                          'expanded_url': 'https://twitter.com/packers/status/1327362434361823232/video/1',
                          'id': 1327340427352174592,
                          'id_str': '1327340427352174592',
                          'indices': [213, 236],
                          'media_url': 'http://pbs.twimg.com/media/EmuqiwVXcAcMirw.jpg',
                          'media_url_https': 'https://pbs.twimg.com/media/EmuqiwVXcAcMirw.jpg',
                          'sizes': {'large': {'h': 720,
                                              'resize': 'fit',
                                              'w': 720},
                                    'medium': {'h': 720,
                                      

           'profile_link_color': '2A433A',
           'profile_sidebar_border_color': 'FFFFFF',
           'profile_sidebar_fill_color': 'DDFFCC',
           'profile_text_color': '333333',
           'profile_use_background_image': False,
           'protected': False,
           'screen_name': 'packers',
           'statuses_count': 66615,
           'time_zone': None,
           'translator_type': 'none',
           'url': 'http://t.co/SOkNpLbeMb',
           'utc_offset': None,
           'verified': True}},
 {'contributors': None,
  'coordinates': None,
  'created_at': 'Fri Nov 13 18:30:00 +0000 2020',
  'display_text_range': [0, 264],
  'entities': {'hashtags': [{'indices': [242, 253], 'text': 'HOFForever'}],
               'media': [{'display_url': 'pic.twitter.com/DDYI6d04h2',
                          'expanded_url': 'https://twitter.com/ProFootballHOF/status/1327317809945325569/video/1',
                          'id': 1327312635180834816,
                          'id_str': 

                       'extended_entities': {'media': [{'additional_media_info': {'description': '',
                                                                                  'embeddable': True,
                                                                                  'monetizable': False,
                                                                                  'title': ''},
                                                        'display_url': 'pic.twitter.com/0w59ckLmuF',
                                                        'expanded_url': 'https://twitter.com/packers/status/1327628046644490241/video/1',
                                                        'id': 1326259298645790722,
                                                        'id_str': '1326259298645790722',
                                                        'indices': [76, 99],
                                                        'media_url': 'http://pbs.twimg.com/amplify_video_thum

  'source': '<a href="http://twitter.com/download/iphone" '
            'rel="nofollow">Twitter for iPhone</a>',
  'team': 'Packers',
  'truncated': False,
  'user': {'contributors_enabled': False,
           'created_at': 'Tue Apr 26 01:34:53 +0000 2016',
           'default_profile': True,
           'default_profile_image': False,
           'description': '@Packers',
           'entities': {'description': {'urls': []},
                        'url': {'urls': [{'display_url': 'twitch.tv/mooggli',
                                          'expanded_url': 'http://twitch.tv/mooggli',
                                          'indices': [0, 23],
                                          'url': 'https://t.co/k80J8RDgXj'}]}},
           'favourites_count': 2517,
           'follow_request_sent': None,
           'followers_count': 289,
           'following': None,
           'friends_count': 160,
           'geo_enabled': True,
           'has_extended_profile': True,
           'id': 72

Link to understand [tweet-object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

Let'see some tweets

In [13]:
for status in tweets[0:max_tweet_to_print]:
    print(status['full_text'] + '\n')

- Heisman Trophy winner (1956)
- No. 1 overall draft pick (1957)
- NFL MVP (1961)
- 4-time NFL champion 
- 1960s NFL All-Decade Team
- Pro Football Hall of Famer (1986)

Remembering #Packers legend Paul Hornung​. https://t.co/gooxAF2Pkc

Paul Hornung, described by Vince Lombardi as "the greatest player I ever coached," has passed away at age 84.

RIP, Golden Boy. https://t.co/jEFkgcGv9T

The entire Pro Football HOF family mourns the passing of Paul Hornung. He was an outstanding player and an incredible man. Known as "The Golden Boy," Paul was above all a leader to whom the Packers looked for the big plays in the big games.

#HOFForever | @packers https://t.co/DDYI6d04h2

RT @packers: To all the true heroes, we say thank you. 

#SaluteToService | @Showtyme_33 https://t.co/0w59ckLmuF



# Pandas DataFrame for storing tweet feed


# Printing keys in tweets

In [14]:
print(tweets[1].keys())
print(len(tweets[1].keys()))


dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang', 'team'])
27


In [15]:
import pandas as pd

# Generators for building DataFrames

In [16]:
def tweet_generator(tweets):
    for tweet in tweets:
        modified_tweet= {}
        
        #Write your code here

        modified_tweet = deepcopy(tweet)
        try:
            modified_tweet["user_id"] = modified_tweet["user"]["id"]
            modified_tweet.pop("user", None)
        except KeyError:
            pass
        try:
            modified_tweet["retweeted_status_id"] = modified_tweet["retweeted_status"]["id"] 
            modified_tweet.pop("retweeted_status", None)
        except KeyError:
            pass
        yield modified_tweet       
            
            

In [17]:
# Checking  generator output
list(tweet_generator(tweets[0:2]))

[{'created_at': 'Fri Nov 13 21:27:20 +0000 2020',
  'id': 1327362434361823232,
  'id_str': '1327362434361823232',
  'full_text': '- Heisman Trophy winner (1956)\n- No. 1 overall draft pick (1957)\n- NFL MVP (1961)\n- 4-time NFL champion \n- 1960s NFL All-Decade Team\n- Pro Football Hall of Famer (1986)\n\nRemembering #Packers legend Paul Hornung\u200b. https://t.co/gooxAF2Pkc',
  'truncated': False,
  'display_text_range': [0, 212],
  'entities': {'hashtags': [{'text': 'Packers', 'indices': [182, 190]}],
   'symbols': [],
   'user_mentions': [],
   'urls': [],
   'media': [{'id': 1327340427352174592,
     'id_str': '1327340427352174592',
     'indices': [213, 236],
     'media_url': 'http://pbs.twimg.com/media/EmuqiwVXcAcMirw.jpg',
     'media_url_https': 'https://pbs.twimg.com/media/EmuqiwVXcAcMirw.jpg',
     'url': 'https://t.co/gooxAF2Pkc',
     'display_url': 'pic.twitter.com/gooxAF2Pkc',
     'expanded_url': 'https://twitter.com/packers/status/1327362434361823232/video/1',
     't

In [18]:
football_feed_df = pd.DataFrame(tweet_generator(tweets))

# Text Cleanup

Here, we're going to take to start cleaning the text in the tweets.

In [19]:
print("Rows with either null text of teams: {}".format(football_feed_df[["full_text","team"]].isnull().any(axis = 1).sum())) 

Rows with either null text of teams: 600


In [20]:
def NoneToNan(elem):
    if elem == "None" or elem == None:
        return np.nan
    else:
        return elem

football_feed_df = football_feed_df.applymap(NoneToNan)

In [21]:
football_feed_df[["full_text","team", "text"]]

,full_text,team,text
0,- Heisman Trophy winner (1956)\n- No. 1 overal...,Packers,NaN
1,"Paul Hornung, described by Vince Lombardi as ""...",Packers,NaN
2,The entire Pro Football HOF family mourns the ...,Packers,NaN
3,"RT @packers: To all the true heroes, we say th...",Packers,NaN
4,@PFF_Packers Domino’s delivery guy to my house.,Packers,NaN
...,...,...,...
1195,NaN,Chiefs,@AndrewKoz88 @Jchrizzap @NFLonFOX @Seahawks @p...
1196,NaN,Chiefs,@BaldyNFL @nyjets @BigTicket73 @Chiefs Thanks ...
1197,NaN,Chiefs,@FaZeJev I feel it jev I love you https://t.co...
1198,NaN,Chiefs,RT @AnaCabrera: Chairman of the Joint Chiefs o...


In [22]:
football_feed_df.full_text.fillna(football_feed_df.text, inplace=True)

football_feed_df[["full_text","team", "text"]]

,full_text,team,text
0,- Heisman Trophy winner (1956)\n- No. 1 overal...,Packers,NaN
1,"Paul Hornung, described by Vince Lombardi as ""...",Packers,NaN
2,The entire Pro Football HOF family mourns the ...,Packers,NaN
3,"RT @packers: To all the true heroes, we say th...",Packers,NaN
4,@PFF_Packers Domino’s delivery guy to my house.,Packers,NaN
...,...,...,...
1195,@AndrewKoz88 @Jchrizzap @NFLonFOX @Seahawks @p...,Chiefs,@AndrewKoz88 @Jchrizzap @NFLonFOX @Seahawks @p...
1196,@BaldyNFL @nyjets @BigTicket73 @Chiefs Thanks ...,Chiefs,@BaldyNFL @nyjets @BigTicket73 @Chiefs Thanks ...
1197,@FaZeJev I feel it jev I love you https://t.co...,Chiefs,@FaZeJev I feel it jev I love you https://t.co...
1198,RT @AnaCabrera: Chairman of the Joint Chiefs o...,Chiefs,RT @AnaCabrera: Chairman of the Joint Chiefs o...


In [23]:
hashtags = []
user_mentions = []

for row in football_feed_df.itertuples():
    row = row.entities
    try:
        hashtags_in_row = row["hashtags"]
        hashtags.append(hashtags_in_row)
    except:
        hashtags.append('')
    try:
        user_mentions_in_row = row["user_mentions"][0]["name"]
        user_mentions.append(user_mentions_in_row)
    except:
        user_mentions.append('')



In [24]:
football_feed_df["hashtags"] = hashtags
football_feed_df["user_mentions"] = user_mentions
football_feed_df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,metadata,source,...,lang,team,user_id,retweeted_status_id,quoted_status_id,quoted_status_id_str,quoted_status,text,hashtags,user_mentions
0,Fri Nov 13 21:27:20 +0000 2020,1327362434361823232,1327362434361823232,- Heisman Trophy winner (1956)\n- No. 1 overal...,False,"[0, 212]","{'hashtags': [{'text': 'Packers', 'indices': [...","{'media': [{'id': 1327340427352174592, 'id_str...","{'result_type': 'popular', 'iso_language_code'...","<a href=""https://studio.twitter.com"" rel=""nofo...",...,en,Packers,35865630,NaN,NaN,NaN,NaN,NaN,"[{'text': 'Packers', 'indices': [182, 190]}]",
1,Fri Nov 13 19:36:20 +0000 2020,1327334499919642625,1327334499919642625,"Paul Hornung, described by Vince Lombardi as ""...",False,"[0, 151]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'result_type': 'popular', 'iso_language_code'...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,en,Packers,35865630,NaN,NaN,NaN,NaN,NaN,[],
2,Fri Nov 13 18:30:00 +0000 2020,1327317809945325569,1327317809945325569,The entire Pro Football HOF family mourns the ...,False,"[0, 264]","{'hashtags': [{'text': 'HOFForever', 'indices'...","{'media': [{'id': 1327312635180834816, 'id_str...","{'result_type': 'popular', 'iso_language_code'...","<a href=""https://ads.twitter.com"" rel=""nofollo...",...,en,Packers,23632910,NaN,NaN,NaN,NaN,NaN,"[{'text': 'HOFForever', 'indices': [242, 253]}]",Green Bay Packers
3,Sat Nov 14 20:07:06 +0000 2020,1327704633796653064,1327704633796653064,"RT @packers: To all the true heroes, we say th...",False,"[0, 112]","{'hashtags': [{'text': 'SaluteToService', 'ind...","{'media': [{'id': 1326259298645790722, 'id_str...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,en,Packers,724773710549372928,1.327628e+18,NaN,NaN,NaN,NaN,"[{'text': 'SaluteToService', 'indices': [57, 7...",Green Bay Packers
4,Sat Nov 14 20:06:46 +0000 2020,1327704548304150529,1327704548304150529,@PFF_Packers Domino’s delivery guy to my house.,False,"[13, 47]","{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,en,Packers,1314216459892133888,NaN,NaN,NaN,NaN,NaN,[],PFF GB Packers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,Fri Nov 13 00:05:05 +0000 2020,1327039746296541184,1327039746296541184,@AndrewKoz88 @Jchrizzap @NFLonFOX @Seahawks @p...,True,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,en,Chiefs,1171063320616427520,NaN,NaN,NaN,NaN,@AndrewKoz88 @Jchrizzap @NFLonFOX @Seahawks @p...,[],Andrew Kozlowski
1196,Fri Nov 13 00:05:05 +0000 2020,1327039745592012802,1327039745592012802,@BaldyNFL @nyjets @BigTicket73 @Chiefs Thanks ...,True,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",...,en,Chiefs,216280533,NaN,NaN,NaN,NaN,@BaldyNFL @nyjets @BigTicket73 @Chiefs Thanks ...,[],Brian Baldinger
1197,Fri Nov 13 00:04:47 +0000 2020,1327039670358745088,1327039670358745088,@FaZeJev I feel it jev I love you https://t.co...,False,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 1327039662720892928, 'id_str...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,en,Chiefs,830269710495145984,NaN,NaN,NaN,NaN,@FaZeJev I feel it jev I love you https://t.co...,[],Jev
1198,Fri Nov 13 00:04:40 +0000 2020,1327039643586621440,1327039643586621440,RT @AnaCabrera: Chairman of the Joint Chiefs o...,False,NaN,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,"{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" 

In [25]:
# write code here

def clean_tweet(text):  

    url = re.compile('https://.+')
    rt = re.compile("RT")
    user = re.compile('@\S+\s')
 
    #'(RT|@.*\s|https://.+)'

    text = url.sub("", text)
    text = rt.sub("", text)
    text = user.sub("", text)

    return text

clean_tweet("RT .@SteveKingIA says he's unsure about the science of climate change. But you don’t need to be a scientist to take ac… https://t.co/J1CcrQZkB2")

football_feed_df["full_text"] = football_feed_df["full_text"].apply(clean_tweet)

In [26]:

def emoji_retriever(text):
      return [(char, emoji.demojize(char)) for char in text if char in emoji.UNICODE_EMOJI]

football_feed_df["emoji"] = football_feed_df["full_text"].apply(emoji_retriever)

In [27]:
contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'),
(r'haven\'t', 'have not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)\'re', '\g<1> are')]

class contraction_replacer(object):
    def __init__(self, contraction_patterns):
        self._contraction_regexes = [(re.compile(p), replaced_text) for p, replaced_text in contraction_patterns]

    def do_contraction_normalization(self, text):
        for contraction_regex, replaced_text in self._contraction_regexes:
            text = contraction_regex.sub(replaced_text, text)
        return text

def preprocess_text(text):
    
    contraction_patterns = [(r'won\'t', 'will not'), (r'can\'t', 'cannot'),
    (r'haven\'t', 'have not'),
    (r'(\w+)\'ll', '\g<1> will'),
    (r'(\w+)\'re', '\g<1> are')]

    replacer = contraction_replacer(contraction_patterns)
    text = replacer.do_contraction_normalization(text)
    text = nltk.tokenize.word_tokenize(text)

    stop_words = set(stopwords.words('english')) 
    
    # lemmatization
    wnlemma = WordNetLemmatizer()
    lemma_text = []
    for word in text:
        if word in stop_words:
            continue
        lemma_text.append(wnlemma.lemmatize(word))
    lemma_text = " ".join(lemma_text)

    return lemma_text

football_feed_df["processed_text"] = football_feed_df["full_text"].apply(preprocess_text)

In [28]:
#Pickle dataframe
import pickle
football_feed_df.to_pickle('football_feed_df.pkl')

In [29]:
def sentiment_analysis(text):
    text = " ".join(text["processed_text"])
    text = TextBlob(text)

    return text.sentiment.polarity

teams_df = football_feed_df[["processed_text", "team"]].groupby("team")
sentiments = teams_df.apply(sentiment_analysis)

# Sentiment Analysis

Below represents the relative polarity of each team from each team on Twitter.

In [30]:
sentiments

team
Bears      0.096870
Chiefs     0.062429
Cowboys    0.176234
Eagles     0.245052
Packers    0.274715
Raiders    0.102080
dtype: float64

In [31]:
def tweet_user_generator(tweets):
    for tweet in tweets:
        yield tweet["user"]
                 

In [32]:
user_df = pd.DataFrame(tweet_user_generator(tweets))

In [33]:
user_df.head()

,id,id_str,name,screen_name,location,description,url,entities,protected,followers_count,...,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type
0,35865630,35865630,Green Bay Packers,packers,"Green Bay, WI",The official Twitter account of the 13-time Wo...,http://t.co/SOkNpLbeMb,{'url': {'urls': [{'url': 'http://t.co/SOkNpLb...,False,2308839,...,DDFFCC,333333,False,True,False,False,None,None,None,none
1,35865630,35865630,Green Bay Packers,packers,"Green Bay, WI",The official Twitter account of the 13-time Wo...,http://t.co/SOkNpLbeMb,{'url': {'urls': [{'url': 'http://t.co/SOkNpLb...,False,2308839,...,DDFFCC,333333,False,True,False,False,None,None,None,none
2,23632910,23632910,Pro Football Hall of Fame,ProFootballHOF,"ÜT: 40.813595,-81.397324",Official Twitter account of the Pro Football H...,https://t.co/uSSWOUw1Y8,{'url': {'urls': [{'url': 'https://t.co/uSSWOU...,False,221817,...,000000,BDBDBD,True,True,False,False,None,None,None,none
3,724773710549372928,724773710549372928,Moogli,AntRussso,⠀⠀⠀⠀⠀⠀⠀518,@Packers,https://t.co/k80J8RDgXj,{'url': {'urls': [{'url': 'https://t.co/k80J8R...,False,289,...,DDEEF6,333333,True,True,True,False,None,None,None,none
4,1314216459892133888,1314216459892133888,Dashiell,Dashiellllll,,I clean water.,None,{'description': {'urls': []}},False,12,...,DDEEF6,333333,True,True,True,False,None,None,None,none


In [34]:
user_df.to_pickle('user_df.pkl')